In [1]:
import json
import sparse
import pandas as pd
import numpy as np
import scipy.sparse

In [7]:
def load_IDs(fname):
    IDs = pd.read_csv(fname, header=0, names=['ID'])
    IDs.index.name = 'i'
    IDs = IDs.reset_index()
    return IDs

def _get_feature_set(df, X_ALL, IDs_ALL):
    IDs = df.set_index('ID')[[]]
    idx = IDs.join(IDs_ALL.set_index('ID'))
    X = [X_ALL[int(i),:] if not np.isnan(i) else sparse.zeros_like(X_ALL[0,:]) for i in idx.values]
    return sparse.stack(X)


def get_features(df, feature_sets):
    features = []
    feature_names = []
    if 'demog' in feature_sets:
        X_d = df.set_index('hosp_id')[['ID']].join(df_demog).reset_index(drop=True).set_index('ID').loc[df['ID']]
        X_d = sparse.as_coo(X_d.values)
        features.append(X_d)
        feature_names.append(names_demog)
        print('demog - Done')
    if 'vitals' in feature_sets:
        X_v = _get_feature_set(df, X_vitals, IDs_vitals)
        features.append(X_v)
        feature_names.append(names_vitals)
        print('vitals - Done')
    if 'meds' in feature_sets:
        X_m = _get_feature_set(df, X_meds, IDs_meds)
        features.append(X_m)
        feature_names.append(names_meds)
        print('meds - Done')
    if 'labs' in feature_sets:
        X_l = _get_feature_set(df, X_labs, IDs_labs)
        features.append(X_l)
        feature_names.append(names_labs)
        print('labs - Done')
    if 'flow' in feature_sets:
        print('flow', end='')
        X_f = _get_feature_set(df, X_flow, IDs_flow)
        features.append(X_f)
        feature_names.append(names_flow)
        print(' - Done')
    X = sparse.concatenate(features, axis=1).tocsr()
    feature_names = sum(feature_names, [])
    return X, np.array(feature_names)


In [6]:
df_demog = pd.read_csv('sample_output/out_demog/static-features.csv').set_index('hosp_id')
names_demog = list(df_demog.columns)
print('demog - Loaded')

X_vitals = sparse.load_npz('sample_output/out_vitals/X_all.npz')
IDs_vitals = load_IDs('sample_output/out_vitals/X_all.IDs.csv')
names_vitals = json.load(open('metadata/out_vitals/X_all.feature_names.json', 'r'))
print('vitals - Loaded')

demog - Loaded
vitals - Loaded


In [10]:
df_cohort = pd.read_csv('sample_input/windows_map.csv')

In [13]:
X, names = get_features(df_cohort, ['demog', 'vitals'])

demog - Done
vitals - Done


In [26]:
df_features = pd.DataFrame(X.todense(), columns=names, index=df_cohort['ID'])

In [27]:
df_features

,"age_value_(17.999, 47.0]","age_value_(47.0, 59.0]","age_value_(59.0, 67.0]","age_value_(67.0, 76.0]","age_value_(76.0, 102.0]",race_value_African American,race_value_American Indian or Alaska Native,race_value_Asian,race_value_Caucasian,race_value_Native Hawaiian and Other Pacific Islander,race_value_Other,race_value_Patient Refused,race_value_Unknown,sex_value_F,sex_value_M,dbp_mask_0.0,dbp_mask_1.0,"dbp_max_(-0.001, 59.0]","dbp_max_(59.0, 65.0]","dbp_max_(65.0, 72.0]","dbp_max_(72.0, 80.0]","dbp_max_(80.0, 344.0]","dbp_mean_(-0.001, 57.0]","dbp_mean_(57.0, 63.0]","dbp_mean_(63.0, 69.0]","dbp_mean_(69.0, 77.0]","dbp_mean_(77.0, 164.333]","dbp_min_(-8.001, 55.0]","dbp_min_(55.0, 60.0]","dbp_min_(60.0, 67.0]","dbp_min_(67.0, 75.0]","dbp_min_(75.0, 154.0]","dbp_value_(-8.001, 57.0]","dbp_value_(57.0, 63.0]","dbp_value_(63.0, 69.0]","dbp_value_(69.0, 77.0]","dbp_value_(77.0, 344.0]",heartrate_mask_0.0,heartrate_mask_1.0,"heartrate_max_(-0.001, 71.0]","heartrate_max_(101.0, 500.0]","heartrate_max_(71.0, 81.0]","heartrate_max_(81.0, 90.0]","heartrate_max_(90.0, 101.0]","heartrate_mean_(-0.001, 69.5]","heartrate_mean_(69.5, 78.667]","heartrate_mean_(78.667, 87.0]","heartrate_mean_(87.0, 97.5]","heartrate_mean_(97.5, 406.0]","heartrate_min_(-0.001, 67.0]",...,"sbp_min_(141.0, 423.0]","sbp_value_(-0.001, 106.0]","sbp_value_(106.0, 118.0]","sbp_value_(118.0, 130.0]","sbp_value_(130.0, 145.0]","sbp_value_(145.0, 463.0]",spo2_mask_0.0,spo2_mask_1.0,"spo2_max_(-0.001, 94.0]","spo2_max_(94.0, 96.0]","spo2_max_(96.0, 97.0]","spo2_max_(97.0, 99.0]","spo2_max_(99.0, 100.0]","spo2_mean_(-0.001, 94.0]","spo2_mean_(94.0, 95.0]","spo2_mean_(95.0, 96.667]","spo2_mean_(96.667, 98.0]","spo2_mean_(98.0, 100.0]","spo2_min_(-0.001, 93.0]","spo2_min_(93.0, 95.0]","spo2_min_(95.0, 96.0]","spo2_min_(96.0, 98.0]","spo2_min_(98.0, 100.0]","spo2_value_(-0.001, 94.0]","spo2_value_(94.0, 95.0]","spo2_value_(95.0, 97.0]","spo2_value_(97.0, 98.0]","spo2_value_(98.0, 100.0]",temperature_mask_0.0,temperature_mask_1.0,"temperature_max_(0.999, 97.7]","temperature_max_(97.7, 98.1]","temperature_max_(98.1, 98.3]","temperature_max_(98.3, 98.8]","temperature_max_(98.8, 146.8]","temperature_mean_(0.999, 97.7]","temperature_mean_(97.7, 98.1]","temperature_mean_(98.1, 98.3]","temperature_mean_(98.3, 98.7]","temperature_mean_(98.7, 136.4]","temperature_min_(0.999, 97.7]","temperature_min_(97.7, 98.0]","temperature_min_(98.0, 98.2]","temperature_min_(98.2, 98.6]","temperature_min_(98.6, 136.4]","temperature_value_(0.999, 97.7]","temperature_value_(97.7, 98.1]","temperature_value_(98.1, 98.3]","temperature_value_(98.3, 98.7]","temperature_value_(98.7, 146.8]"
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
90000-____0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,...,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1
90000-____1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,...,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1
90001-____0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,...,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1


In [ ]:
# X_meds = sparse.load_npz('/data/covid/process_data/FIDDLE_2020_LATEST/new_meds/X_all.npz')
# IDs_meds = load_IDs('/data/covid/process_data/FIDDLE_2020_LATEST/new_meds/X_all.IDs.csv')
# names_meds = json.load(open('/data/covid/process_data/data-2015-2019/new_meds/X_all.feature_names.json', 'r'))
# print('meds - Loaded')

# X_labs = sparse.load_npz('/data/covid/process_data/FIDDLE_2020_LATEST/out_labs/X_all.npz')
# IDs_labs = load_IDs('/data/covid/process_data/FIDDLE_2020_LATEST/out_labs/X_all.IDs.csv')
# names_labs = json.load(open('/data/covid/model_ICU_care/out_labs/X_all.feature_names.json', 'r'))
# print('labs - Loaded')

# X_flow = sparse.load_npz('/data/covid/process_data/FIDDLE_2020_LATEST/out_flow/X_all.20.npz')
# IDs_flow = load_IDs('/data/covid/process_data/FIDDLE_2020_LATEST/out_flow/X_all.20.IDs.csv')
# names_flow = json.load(open('/data/covid/process_data/NEW-flow/out_flow/X_all.feature_names.json', 'r'))
# print('flow - Loaded')

In [ ]:

# print('Removing specific features')

# # Remove meds features related to: famotidine (154306), LORAZEPAM (154336), PROPOFOL (154447)
# print('Removing meds', end='')
# # meds_remove = [121, 122, 123, 199, 200, 201, 202, 203, 204, 472, 473, 474, 475, 476, 477]
# meds_remove = [119, 120, 121, 206, 207, 208, 209, 210, 211, 212, 213, 509, 510, 511, 512, 513, 514, 515, 516]
# X_meds = scipy.sparse.lil_matrix(X_meds.tocsr())
# X_meds[:, meds_remove] = 0
# X_meds = sparse.as_coo(X_meds)
# print(' - Done')

# # Remove labs: 82989
# print('Removing labs', end='')
# labs_remove = [5546, 5547, 5548, 5549, 5550, 5551, 5552, 5553, 5554, 5555, 5556, 5557, 5558, 5559, 5560, 5561, 5562, 5563, 5564, 5565, 5566, 5567, 5568, 5569]
# X_labs = scipy.sparse.lil_matrix(X_labs.tocsr())
# X_labs[:, labs_remove] = 0
# X_labs = sparse.as_coo(X_labs)
# print(' - Done')

